## Tabla de contenidos 

1. [Contexto del modelo](#id1)

    1.1. [Librerías utilizadas](#id2)

2. [Preparación del dataset](#id3)
    
3. [Construcción del modelo](#id4)

4. [Predicciones del modelo](#id5)

5. [Evaluación](#id6)

    5.1. [Conclusiones](#id7)

6. [Exportación del modelo](#id8)

<div id='id1' />

# 1. Contexto del modelo

El modelo en cuestión es un clasificador de texto que utiliza técnicas de procesamiento de lenguaje natural (NLP) para la tarea de clasificación multietiqueta. Más específicamente, utiliza un vectorizador TF-IDF (Term Frequency-Inverse Document Frequency) junto con un clasificador de regresión logística multinominal.

<div id='id2' />

## 1.1. Librerías utilizadas

In [3]:
# Tratamiento de datos

import numpy as np
import pandas as pd
import pickle
import warnings
warnings.filterwarnings('ignore')

# Gráficas

import seaborn as sns
import matplotlib as mpl 
import matplotlib.pyplot as plt

# Preprocesamiento y modelado

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

<div id='id3' />

# 2. Preparación del dataset

In [4]:
comments_df_undersampled = pd.read_csv('../01_data/02_processed/comments_df_undersampled.csv')
comments_df_undersampled

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,11ec6078ad3cd031,\n\nsee in my userpage the articles ive create...,0,0,0,0,0,0
1,11c0246ef55ef048,\n\nmatt fax page being deleted\n\nhi you just...,0,0,0,0,0,0
2,278ec4e59398a50b,this article contains information soursed from...,0,0,0,0,0,0
3,c2788f8bdaefeb13,24 promo \n\ni was really confused when i saw ...,0,0,0,0,0,0
4,fa04aa41f79c7884,i agree hes a biased editor 1141791837,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
32445,fef4cf7ba0012866,\n\n our previous conversation \n\nyou fucking...,1,0,1,0,1,1
32446,ff39a2895fc3b40e,you are a mischievious pubic hair,1,0,0,0,1,0
32447,ffa33d3122b599d6,your absurd edits \n\nyour absurd edits on gre...,1,0,1,0,1,0
32448,ffb47123b2d82762,\n\nhey listen dont you ever delete my edits e...,1,0,0,0,1,0


In [5]:
comments_df_undersampled.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')

In [6]:
# Realizamos el train-test split

X_train, X_test, y_train, y_test = train_test_split(comments_df_undersampled['comment_text'], comments_df_undersampled[[ 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate']], test_size=0.2, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(25960,)
(6490,)
(25960, 6)
(6490, 6)


<div id='id4' />

# 3. Construcción del modelo

In [7]:
# Instanciamos el TfidfVectorizer y del LogisticRegression multinominal con sus correspondientes parámetros

vectorizer = TfidfVectorizer(stop_words="english", ngram_range=(1,2), max_features=500000)
classifier = MultiOutputClassifier(LogisticRegression(max_iter=10000))

In [8]:
# Generamos el pipeline con las dos instancias anteriores y lo entrenamos

pipeline = Pipeline([
    ('vectorizer', vectorizer),
    ('classifier', classifier)
])

pipeline.fit(X_train, y_train)

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(max_features=500000, ngram_range=(1, 2),
                                 stop_words='english')),
                ('classifier',
                 MultiOutputClassifier(estimator=LogisticRegression(max_iter=10000)))])

<div id='id5' />

# 4. Predicciones del modelo

In [9]:
# Realizamos la predicción

y_pred = pipeline.predict(X_test)

In [11]:
pipeline.predict(['You are a piece of shit, I am going to kill you', 'I love you, you are the best'])

array([[1, 0, 1, 1, 1, 0],
       [0, 0, 0, 0, 0, 0]])

<div id='id6' />

# 5. Evaluación del modelo

In [13]:
# Calculamos la matriz de confusión, precisión, recall y categorical accuracy para cada etiqueta

labels = comments_df_undersampled[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']]

for i, label in enumerate(labels.columns):
    print(f"Resultados para la etiqueta: {label}")
    print(f"Confusion Matrix:\n{confusion_matrix(y_test.iloc[:, i], y_pred[:, i])}")
    print(f"Precision: {precision_score(y_test.iloc[:, i], y_pred[:, i], zero_division=1)}")
    print(f"Recall: {recall_score(y_test.iloc[:, i], y_pred[:, i], zero_division=1)}")
    print(f"Categorical Accuracy: {accuracy_score(y_test.iloc[:, i], y_pred[:, i])}")
    print("-" * 50)

# Calculamos la precisión categórica general, o lo que es lo mismo, que todas las etiquetas deban coincidir

categorical_accuracy = accuracy_score(y_test, y_pred)
print(f"Categorical Accuracy General: {categorical_accuracy}")

Resultados para la etiqueta: toxic
Confusion Matrix:
[[3139  298]
 [ 529 2524]]
Precision: 0.8944011339475549
Recall: 0.8267278087127415
Categorical Accuracy: 0.8725731895223421
--------------------------------------------------
Resultados para la etiqueta: severe_toxic
Confusion Matrix:
[[6121   71]
 [ 233   65]]
Precision: 0.47794117647058826
Recall: 0.2181208053691275
Categorical Accuracy: 0.9531587057010786
--------------------------------------------------
Resultados para la etiqueta: obscene
Confusion Matrix:
[[4671  115]
 [ 566 1138]]
Precision: 0.9082202713487629
Recall: 0.6678403755868545
Categorical Accuracy: 0.8950693374422188
--------------------------------------------------
Resultados para la etiqueta: threat
Confusion Matrix:
[[6395    6]
 [  76   13]]
Precision: 0.6842105263157895
Recall: 0.14606741573033707
Categorical Accuracy: 0.9873651771956856
--------------------------------------------------
Resultados para la etiqueta: insult
Confusion Matrix:
[[4663  271]
 [ 65

In [14]:
# Calcular y mostrar la precisión y el recall combinados

precision_micro = precision_score(y_test, y_pred, average='micro', zero_division=1)
recall_micro = recall_score(y_test, y_pred, average='micro', zero_division=1)
f1_micro = f1_score(y_test, y_pred, average='micro', zero_division=1)

precision_macro = precision_score(y_test, y_pred, average='macro', zero_division=1)
recall_macro = recall_score(y_test, y_pred, average='macro', zero_division=1)
f1_macro = f1_score(y_test, y_pred, average='macro', zero_division=1)

print(f"Precision (Micro): {precision_micro}")
print(f"Recall (Micro): {recall_micro}")
print(f"F1 Score (Micro): {f1_micro}")

print(f"Precision (Macro): {precision_macro}")
print(f"Recall (Macro): {recall_macro}")
print(f"F1 Score (Macro): {f1_macro}")

# Calcular y mostrar la precisión categórica general (es decir, todas las etiquetas deben coincidir)

categorical_accuracy = accuracy_score(y_test, y_pred)
print(f"Categorical Accuracy General: {categorical_accuracy}")

Precision (Micro): 0.858085808580858
Recall (Micro): 0.672607070997413
F1 Score (Micro): 0.7541089268449888
Precision (Macro): 0.7494312335956059
Recall (Macro): 0.4331113209492645
F1 Score (Macro): 0.5161110248188719
Categorical Accuracy General: 0.6583975346687211


<div id='id7' />

## 5.1. Conclusión

Este modelo consiste en una iteración sobre el segundo de los modelos construidos. La única diferencia entre ambos es que éste incluye un parámetro dentro de la vectorización de los comentarios que hace que se eliminen las palabras más frecuentes dentro del inglés. En cualquier caso, tras observar los resultados de la evaluación podemos ver que apenas hay diferencias en los resultados obtenidos por lo que podemos afirmar que esta modificación en el set de datos tiene un impacto mínimo en los resultados que nos proporciona el modelo.

<div id='id8' />

# 6. Exportación del modelo

In [25]:
# Definimos el nombre del archivo
filename = '03_segundo_modelo.pkl'

# Guardamos el pipeline en un archivo pickle

with open(filename, 'wb') as file:
    pickle.dump(pipeline, file)